Here's an example with an alternative GUI.

https://stackoverflow.com/questions/15460706/opencv-cv2-in-python-videocapture-not-releasing-camera-after-deletion

In [1]:
from collections import deque
import cv2
import time
import tkinter as tk
from PIL import Image, ImageTk

In [2]:
from src.sources.polar_mapping import generateLUT, img2polar
import numpy as np

In [3]:
def display(image_array, label):
    """Convert image array to image for display.
    
    Args:
        image_array - 2D numpy array.
        label - tkinter label for display.
    Returns:
        la
    """
    a = Image.fromarray(image_array)
    b = ImageTk.PhotoImage(image=a)
    label.configure(image=b)
    label._image_cache = b

Can we create sub-classes for each GUI component?

In [14]:
class CameraGUI:
    
    def __init__(self, src=0):
        # Set Up Camera and LUT for Polar Mapping
        self.cam = cv2.VideoCapture(src)
        self.cam.set(cv2.CAP_PROP_CONVERT_RGB, 0)
        # Capture a frame to set image sizes
        _, frame = self.cam.read()
        Y = frame[:, :, 0]
        self.centre = np.asarray(Y.shape) // 2
        self.final_radius = self.centre.min()
        # Generate LUT
        self.LUT = generateLUT(self.centre, self.final_radius, phase_width=256)
        # Setup gui
        self.window = tk.Tk()
        # label for the original video frame
        self.original_image = tk.Label(master=self.window)
        self.original_image.grid(row=0, column=0)
        # label for the polar video frame
        self.polar_image = tk.Label(master=self.window)
        self.polar_image.grid(row=0, column=1)
        # Label for right image
        self.right_image = tk.Label(master=self.window)
        self.right_image.grid(row=0, column=2)
        # Label for left image
        self.left_image = tk.Label(master=self.window)
        self.left_image.grid(row=0, column=3)
        
        # label for fps
        self.fps_label = tk.Label(master=self.window)
        self.fps_label.grid(row=2, column=0)
        self.fps_label._frame_times = deque([0]*5)  # arbitrary 5 frame average FPS
        # quit button
        self.quit_button = tk.Button(master=self.window, text='Quit', command=lambda: self.quit_())
        self.quit_button.grid(row=2, column=1)
        # setup the update callback
        self.window.after(0, func=lambda: self.update_all())
    
    def update_image(self):
        # Get frame
        (readsuccessful, frame) = self.cam.read()
        Y = frame[:, :, 0]
        # Show original frame
        display(cv2.pyrDown(Y), self.original_image)
        # Can change this to internal method
        converted = img2polar(Y, self.centre, self.final_radius, self.LUT, phase_width=256)
        display(converted, self.polar_image)
        # Show left / right hemisphere images
        rows, cols = converted.shape
        # right_image = np.flipud(converted[:, :cols//2].T)
        # left_image = np.fliplr(converted[:, cols//2:].T)
        right_image = converted[:, :cols//2].T
        left_image = np.flip(converted[:, cols//2:].T)
        display(right_image, self.right_image)
        display(left_image, self.left_image)
        # Update Window
        self.window.update()
    
    def update_fps(self):
        frame_times = self.fps_label._frame_times
        frame_times.rotate()
        frame_times[0] = time.time()
        sum_of_deltas = frame_times[0] - frame_times[-1]
        count_of_deltas = len(frame_times) - 1
        try:
            fps = int(float(count_of_deltas) / sum_of_deltas)
        except ZeroDivisionError:
            fps = 0
        self.fps_label.configure(text=f'FPS: {fps}')
        
    def update_all(self):
        self.update_image()
        self.update_fps()
        self.window.after(20, func=lambda: self.update_all())
    
    def run(self):
        self.window.mainloop()
        
    def quit_(self):
        self.cam.release()
        self.window.destroy()

In [16]:
camGUI = CameraGUI(src=2)

camGUI.run()

In [ ]:
from src.sources.polar_mapping import 

In [ ]:
class CamGUIReduced(CameraGUI):
    
    def __init__(self, src=0):
        # Set Up Camera and LUT for Polar Mapping
        self.cam = cv2.VideoCapture(src)
        self.cam.set(cv2.CAP_PROP_CONVERT_RGB, 0)
        # Capture a frame to set image sizes
        _, frame = self.cam.read()
        Y = frame[:, :, 0]
        self.centre = np.asarray(Y.shape) // 2
        self.final_radius = self.centre.min()
        # Generate LUT
        self.LUT = generateLUT(self.centre, self.final_radius, phase_width=256)
        # Setup gui
        self.window = tk.Tk()
        # label for the original video frame
        self.original_image = tk.Label(master=self.window)
        self.original_image.grid(row=0, column=0)
        # label for the polar video frame
        self.polar_image = tk.Label(master=self.window)
        self.polar_image.grid(row=0, column=1)
        # Label for right image
        self.right_image = tk.Label(master=self.window)
        self.right_image.grid(row=0, column=2)
        # Label for left image
        self.left_image = tk.Label(master=self.window)
        self.left_image.grid(row=0, column=3)
        
        # label for fps
        self.fps_label = tk.Label(master=self.window)
        self.fps_label.grid(row=2, column=0)
        self.fps_label._frame_times = deque([0]*5)  # arbitrary 5 frame average FPS
        # quit button
        self.quit_button = tk.Button(master=self.window, text='Quit', command=lambda: self.quit_())
        self.quit_button.grid(row=2, column=1)
        # setup the update callback
        self.window.after(0, func=lambda: self.update_all())
    
    def update_image(self):
        # Get frame
        (readsuccessful, frame) = self.cam.read()
        Y = frame[:, :, 0]
        # Show original frame
        display(cv2.pyrDown(Y), self.original_image)
        # Can change this to internal method
        converted = img2polar(Y, self.centre, self.final_radius, self.LUT, phase_width=256)
        display(converted, self.polar_image)
        # Show left / right hemisphere images
        rows, cols = converted.shape
        # right_image = np.flipud(converted[:, :cols//2].T)
        # left_image = np.fliplr(converted[:, cols//2:].T)
        right_image = converted[:, :cols//2].T
        left_image = np.flip(converted[:, cols//2:].T)
        display(right_image, self.right_image)
        display(left_image, self.left_image)
        # Update Window
        self.window.update()
    
    def update_fps(self):
        frame_times = self.fps_label._frame_times
        frame_times.rotate()
        frame_times[0] = time.time()
        sum_of_deltas = frame_times[0] - frame_times[-1]
        count_of_deltas = len(frame_times) - 1
        try:
            fps = int(float(count_of_deltas) / sum_of_deltas)
        except ZeroDivisionError:
            fps = 0
        self.fps_label.configure(text=f'FPS: {fps}')
        
    def update_all(self):
        self.update_image()
        self.update_fps()
        self.window.after(20, func=lambda: self.update_all())
    
    def run(self):
        self.window.mainloop()
        
    def quit_(self):
        self.cam.release()
        self.window.destroy()